In [15]:
import pandas as pd
import numpy as np

#nlp
import spacy
import re
from textblob import TextBlob

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords 

#LDA / topical modeling
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

from nltk.corpus import stopwords
stop = stopwords.words('english')

#Visualizations
import plotly

In [16]:
stop += ['rt']

## Data Processing

In [3]:
def add_datepart(df, fldname, drop=True, time=False):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.
    Examples:
    ---------
    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df
        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13
    >>> add_datepart(df, 'A')
    >>> df
        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    """
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [4]:
tweets = pd.read_csv('../../finalprojdata/tweets.csv')
users = pd.read_csv('../../finalprojdata/users.csv')

In [5]:
fulltweets = tweets.merge(users, how='left', left_on='user_key', right_on='screen_name')

In [6]:
add_datepart(fulltweets, 'created_str')

## NLP

In [15]:
## Create a sentiment column

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analyze_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

In [18]:
tweetsonly2 = fulltweets.text.copy().astype(str)
tweetsonly2 = tweetsonly2.str.replace('[^\w\s]','')
tweetsonly2 = tweetsonly2.str.replace('[\\r|\\n|\\t|_]',' ')
tweetsonly2 = tweetsonly2.str.strip()
tweetsonly2

fulltweets2 = fulltweets.copy()
fulltweets2.text = tweetsonly2

In [19]:
fulltweets2.text = fulltweets2.text.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop)]))
fulltweets2['Sentiment'] = np.array([analyze_sentiment(str(tweet)) for tweet in fulltweets2.text.values])

fulltweets2.text = fulltweets2.text.apply(lambda x: ' '.join([word.lower() for word in x.split() if len(word) > 3]))

In [20]:
fulltweets2.head()

,user_id,user_key,created_at_x,retweet_count,retweeted,favorite_count,text,tweet_id,source,hashtags,...,friends_count,listed_count,created_strYear,created_strMonth,created_strWeek,created_strDay,created_strDayofweek,created_strDayofyear,created_strElapsed,Sentiment
0,1.868981e+09,ryanmaxwell_1,1.458672e+12,NaN,NaN,NaN,islamkills trying terrorist attacks europe ref...,7.123460e+17,NaN,"[""IslamKills""]",...,NaN,NaN,2016.0,3.0,12.0,22.0,1.0,82.0,1458671502,0.000000
1,2.571870e+09,detroitdailynew,1.476133e+12,0.0,False,0.0,clinton trump shouldve apologized attacked les...,7.855849e+17,"<a href=""http://twitterfeed.com"" rel=""nofollow...",[],...,NaN,NaN,2016.0,10.0,41.0,10.0,0.0,284.0,1476133020,-0.166667
2,1.710805e+09,cookncooks,1.487767e+12,NaN,NaN,NaN,ltapoll wasis best president past years vote r...,8.343832e+17,NaN,[],...,NaN,NaN,2017.0,2.0,8.0,22.0,2.0,53.0,1487767423,0.375000
3,2.584153e+09,queenofthewo,1.482765e+12,NaN,NaN,NaN,jww372 dont guess religion christmasaftermath,8.134006e+17,NaN,"[""ChristmasAftermath""]",...,1427.0,11.0,2016.0,12.0,52.0,26.0,0.0,361.0,1482764801,0.000000
4,1.768260e+09,mrclydepratt,1.501987e+12,NaN,NaN,NaN,shareblue pence lawyers decided official email...,8.940243e+17,NaN,[],...,847.0,23.0,2017.0,8.0,31.0,6.0,6.0,218.0,1501986984,0.000000


In [21]:
fulltweets2.to_csv('/Users/shsu/Downloads/fulltweets.csv')

## spacy implementation

https://nlpforhackers.io/complete-guide-to-spacy/

In [9]:
nlp = spacy.load('en')

## LDA 
https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

might be too big of data:

https://turi.com/products/create/docs/graphlab.toolkits.text_analytics.html 

In [77]:
fulltweets2.text.values

array(['islamkills trying terrorist attacks europe refugees',
       'clinton trump shouldve apologized attacked less httpstcoejampkohfz',
       'ltapoll wasis best president past years vote retweet', ...,
       'signsinyork getting right company logo businesses message across smallbusinessuk httpstcofnse5ga6dc',
       'latest obama affirms continuity ties canada httpstcoq4bznwxdss httpstcogpy8dgkiff',
       'futureguru100 cant upload online thats product worku gotta represent brand qual'], dtype=object)

In [24]:
# compile sample documents into a list
doc_set = fulltweets2.text.values.copy()

# list for tokenized documents in loop
#texts = np.array(['']*fulltweets2.shape[0])

# loop through document list
texts = [text.split(' ') for text in doc_set]

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary)

In [33]:
print(ldamodel.print_topics(num_topics=10, num_words=10))

[(23, '0.060*"america" + 0.047*"make" + 0.031*"great" + 0.029*"today" + 0.014*"sure" + 0.013*"done" + 0.012*"trump2016" + 0.012*"join" + 0.011*"plan" + 0.011*"realdonaldtrump"'), (24, '0.051*"dont" + 0.040*"know" + 0.028*"trump" + 0.027*"going" + 0.027*"white" + 0.021*"house" + 0.018*"support" + 0.018*"want" + 0.016*"people" + 0.015*"come"'), (20, '0.133*"trump" + 0.048*"donald" + 0.036*"news" + 0.032*"trumps" + 0.023*"clinton" + 0.022*"election" + 0.022*"president" + 0.020*"hillary" + 0.019*"politics" + 0.017*"media"'), (3, '0.028*"real" + 0.013*"https" + 0.011*"shows" + 0.011*"reason" + 0.010*"polls" + 0.009*"thingsmoretrustedthanhillary" + 0.009*"realdonaldtrump" + 0.009*"trump" + 0.009*"protect" + 0.008*"thehill"'), (13, '0.021*"russia" + 0.018*"rally" + 0.016*"getting" + 0.014*"talk" + 0.011*"fire" + 0.010*"christmasaftermath" + 0.010*"using" + 0.010*"florida" + 0.009*"hear" + 0.009*"tweets"'), (7, '0.086*"hillary" + 0.075*"clinton" + 0.050*"tcot" + 0.036*"pjnet" + 0.022*"bill" + 

In [2]:
!pip install pyldavis
#!pip install graphlab-create

    100% |████████████████████████████████| 184kB 3.2MB/s ta 0:00:01
    100% |████████████████████████████████| 174kB 3.1MB/s ta 0:00:011
  Running setup.py bdist_wheel for funcy ... done
  Stored in directory: /Users/shsu/Library/Caches/pip/wheels/e8/f5/5a/b4220be9e3fcc3e63df44253b9e1483aee24ec8e63b9da69ea
Successfully built funcy
You are using pip version 9.0.2, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Visualize topics

In [6]:
fulltweet = pd.read_csv('/Users/shsu/Downloads/fulltweets.csv')

In [17]:
#p_stemmer = PorterStemmer()

# compile sample documents into a list
doc_set = fulltweet.text.astype(str).values.copy()

# list for tokenized documents in loop
#texts = np.array(['']*fulltweets2.shape[0])

# loop through document list
texts = [text.split(' ') for text in doc_set]

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=30, id2word = dictionary)

In [43]:
dictionary.save('dictionary.dict')

corpora.MmCorpus.serialize('corpus.mm', corpus)

ldamodel.save('topic.model')


d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

In [18]:
import pandas as pd
import pyLDAvis
import pyLDAvis.graphlab

pyLDAvis.enable_notebook()

In [22]:
fulltweet.head()

,Unnamed: 0,user_id,user_key,created_at_x,retweet_count,retweeted,favorite_count,text,tweet_id,source,...,friends_count,listed_count,created_strYear,created_strMonth,created_strWeek,created_strDay,created_strDayofweek,created_strDayofyear,created_strElapsed,Sentiment
0,0,1.868981e+09,ryanmaxwell_1,1.458672e+12,NaN,NaN,NaN,islamkills trying terrorist attacks europe ref...,7.123460e+17,NaN,...,NaN,NaN,2016.0,3.0,12.0,22.0,1.0,82.0,1458671502,0.000000
1,1,2.571870e+09,detroitdailynew,1.476133e+12,0.0,False,0.0,clinton trump shouldve apologized attacked les...,7.855849e+17,"<a href=""http://twitterfeed.com"" rel=""nofollow...",...,NaN,NaN,2016.0,10.0,41.0,10.0,0.0,284.0,1476133020,-0.166667
2,2,1.710805e+09,cookncooks,1.487767e+12,NaN,NaN,NaN,ltapoll wasis best president past years vote r...,8.343832e+17,NaN,...,NaN,NaN,2017.0,2.0,8.0,22.0,2.0,53.0,1487767423,0.375000
3,3,2.584153e+09,queenofthewo,1.482765e+12,NaN,NaN,NaN,jww372 dont guess religion christmasaftermath,8.134006e+17,NaN,...,1427.0,11.0,2016.0,12.0,52.0,26.0,0.0,361.0,1482764801,0.000000
4,4,1.768260e+09,mrclydepratt,1.501987e+12,NaN,NaN,NaN,shareblue pence lawyers decided official email...,8.940243e+17,NaN,...,847.0,23.0,2017.0,8.0,31.0,6.0,6.0,218.0,1501986984,0.000000


In [38]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
data

/Library/Python/2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
17     5.650816        1       1  0.269390 -0.038418
21     4.081374        1       2  0.207074 -0.011219
28     4.004640        1       3 -0.044191  0.224570
27     3.965590        1       4  0.087466  0.131443
9      3.849046        1       5  0.190262  0.006419
25     3.681144        1       6  0.166511 -0.017891
11     3.527789        1       7 -0.027825  0.066462
14     3.455561        1       8  0.083642 -0.104792
24     3.438317        1       9 -0.027985  0.111117
4      3.321583        1      10 -0.078400  0.164542
0      3.231107        1      11 -0.017994 -0.076229
13     3.229781        1      12  0.058620 -0.002203
23     3.223659        1      13 -0.023846  0.128801
12     3.188598        1      14 -0.003502  0.105461
7      3.183856        1      15 -0.043826 -0.092018
19     3.098103        1      16 -0.062619 -0.077927
8      3.086215        1      17 -0.027137 -0.051963
29     3.080216        1      18 -0.049124 -0.019002
3      3.076744        1      19  0.050652  0.003056
16     3.033913        1      20 -0.097865 -0.055391
10     3.015156        1      21 -0.080050 -0.095464
22     3.013438        1      22 -0.038976 -0.059701
20     3.003510        1      23 -0.085357 -0.049674
5      2.998346        1      24 -0.072599 -0.006369
15     2.983902        1      25 -0.050851 -0.035763
1      2.951789        1      26 -0.080618 -0.057125
18     2.947258        1      27 -0.084379  0.010517
2      2.922697        1      28 -0.059199 -0.082429
6      2.913005        1      29 -0.059057  0.003384
26     2.842849        1      30  0.001783 -0.022193, topic_info=       Category          Freq             Term         Total  loglift  logprob
term                                                                          
235874  Default  23803.000000            trump  23803.000000  30.0000  30.0000
31953   Default   9605.000000          clinton   9605.000000  29.0000  29.0000
172447  Default   9015.000000          hillary   9015.000000  28.0000  28.0000
38966   Default   4988.000000             dont   4988.000000  27.0000  27.0000
48151   Default   3587.000000             tcot   3587.000000  26.0000  26.0000
56046   Default   4470.000000           donald   4470.000000  25.0000  25.0000
27477   Default   3918.000000         politics   3918.000000  24.0000  24.0000
183588  Default   3519.000000          america   3519.000000  23.0000  23.0000
36914   Default   3093.000000        president   3093.000000  22.0000  22.0000
10757   Default   4861.000000             like   4861.000000  21.0000  21.0000
238384  Default   2892.000000             know   2892.000000  20.0000  20.0000
135144  Default   2796.000000             vote   2796.000000  19.0000  19.0000
22149   Default   2609.000000            never   2609.000000  18.0000  18.0000
233129  Default   2504.000000             good   2504.000000  17.0000  17.0000
119324  Default   2668.000000             want   2668.000000  16.0000  16.0000
223723  Default   2392.000000            white   2392.000000  15.0000  15.0000
136538  Default   2598.000000            pjnet   2598.000000  14.0000  14.0000
74981   Default   2431.000000             love   2431.000000  13.0000  13.0000
66034   Default   2583.000000             maga   2583.000000  12.0000  12.0000
176039  Default   2245.000000         midnight   2245.000000  11.0000  11.0000
231205  Default   2439.000000            think   2439.000000  10.0000  10.0000
7915    Default   2239.000000             cant   2239.000000   9.0000   9.0000
136102  Default   7363.000000            obama   7363.000000   8.0000   8.0000
30154   Default   2032.000000             back   2032.000000   7.0000   7.0000
35015   Default   1937.000000         american   1937.000000   6.0000   6.0000
190213  Default   4275.000000             news   4275.000000   5.0000   5.0000
216484  Default   2734.000000             make   

## Save the entire html

In [40]:
pyLDAvis.save_html(data, 'russian.html')